In [8]:
import pandas as pd
# if you want to download the original file:
#df = pd.read_csv('https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/50k_imdb_movie_reviews.csv')
# otherwise load local file
df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


# Definition function to process text:

In [9]:
# Return a lower case proccesed text
def processtext(texto):
    import re
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\n)")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    texto = REPLACE_NO_SPACE.sub('', texto.lower())
    texto = REPLACE_WITH_SPACE.sub(' ', texto)
    return texto

## Get the number of pronouns in a text:

In [29]:
# Get the number of 1st and 2nd pronouns:
def numberpronouns(texto):
    import re
    PRONOUNS = [r'\bi\b',r'\bwe\b',r'\byou\b',r'\bme\b',r'\bmyself\b',r'\byourself\b',r'\bll\b']
    L = [re.findall(pro, texto) for pro in PRONOUNS]
    lengh = 0
    for l in L:
        lengh = lengh + len(l)
    return lengh

## Is if No, not, neither word is contained

In [13]:
def isNoThere(texto):
    import re
    NO_VARS = [r'\bno\b',r'\bnot\b',r'\bneither\b']
    L_NO = [re.findall(neg, texto) for neg in NO_VARS]
    le = 0
    for ll in L_NO:
        le = le+len(ll)
    if(le > 0):
        return True
    else:
        return False

## Is ( ! ) character contained in the text?

In [14]:
def isExclamationThere(texto):
    import re
    EX_VAR = r'!'
    L_EXC = re.findall(EX_VAR, texto)
    if(len(L_EXC) > 0):
        return True
    else:
        return False

## Get the logarithm of number of words

In [22]:
def getLnNumberWords(texto_dividido):
    import numpy as np
    N_WORDS = len(texto_dividido)
    return np.log(N_WORDS)

## Get Number of Positive and negative words

In [20]:
def getPositiveNegativeCountWords(texto_dividido, posneg_dictionary):
        # Count the positive words
    COUNT_POSITIVE = 0
    COUNT_NEGATIVE = 0
    for word in texto_dividido:
        try:
            val = posneg_dictionary[word]
            if val == 1:
                COUNT_POSITIVE = COUNT_POSITIVE + 1
            elif val == 0:
                COUNT_NEGATIVE = COUNT_NEGATIVE + 1

        except KeyError:
            pass
    
    return (COUNT_POSITIVE, COUNT_NEGATIVE)

## Chargue full dictionary:

In [19]:
def chargeDictionariPosNeg():
    posneg_dictionary = {}
    # Charge positive file of words to dictionary
    with open('goodbad/positive-words.txt','r') as fichero:
        import re
        for lin in fichero:
            if('a+' in lin):
                break
        for lin in fichero:
            if(' ' not in lin):
                posneg_dictionary[re.sub('\n','',lin)] = 1
                
    # Charge negative file of words to dictionary
    with open('goodbad/negative-words.txt','r') as fichero:
        import re
        for lin in fichero:
            if('2-faces' in lin):
                break
        for lin in fichero:
            if(' ' not in lin):
                posneg_dictionary[re.sub('\n','',lin)] = 0
    
    return posneg_dictionary

# Extract full features

In [25]:
dictionary = chargeDictionariPosNeg()

In [24]:
def extract_features(texto, dictionary):
    import re
    texto = processtext(texto)
    texto_dividido = re.split(' ', texto)
    x1, x2 = getPositiveNegativeCountWords(texto_dividido, dictionary)
    x3 = isNoThere(texto)
    x4 = numberpronouns(texto)
    x5 = isExclamationThere(texto)
    x6 = getLnNumberWords(texto_dividido)
    return (x1, x2, x3, x4, x5, x6)

In [26]:
texto = """In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70's, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich family used their influence to cover the murder for more than twenty years. However, a snoopy detective and convicted perjurer in disgrace was able to disclose how the hideous crime was committed. The screenplay shows the investigation of Mark and the last days of Martha in parallel, but there is a lack of the emotion in the dramatization. My vote is seven.<br /><br />Title (Brazil): Not Available
"""

In [30]:
M = extract_features(texto, dictionary)

In [31]:
M

(8, 14, True, 0, False, 5.472270673671475)